In [1]:
# change the cwd
import os
os.chdir(os.getcwd()+"/HLP_Project/")

In [2]:
# use cython compiled modules for faster execution
from libs.cython.pyx.edgelistParser import edgelistParser
from libs.cython.pyx.utils import list2matrix, getInOutDegree
from libs.cython.cpp.pageRankCpp import cppLocalPageRank as localPageRank
from libs.cython.cpp.pageRankCpp import cppLocalPageRank_weight as localPageRank_weight

In [3]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
import datetime

In [4]:
_ = """
Dataset Types
../datasets/facebook/*.edges -> raw_list
../datasets/facebook_combined.txt -> list
../datasets/p2p-Gnutella31.txt -> list
../datasets/Wiki-Vote.txt -> list
../datasets/CA-HepPh.txt -> raw_list
../datasets/com-dblp.ungraph.txt -> list
../datasets/Email-Enron.txt -> raw_list
../datasets/Email-EuAll.txt -> list
"""

In [5]:
_ = """
../datasets/facebook/0.edges -> N = 333
../datasets/facebook/107.edges -> N = 1034
../datasets/facebook/348.edges -> N = 224
../datasets/facebook/414.edges -> N = 150
../datasets/facebook/686.edges -> N = 168
../datasets/facebook/698.edges -> N = 61
../datasets/facebook/1684.edges -> N = 786
../datasets/facebook/1912.edges -> N = 747
../datasets/facebook/3437.edges -> N = 534
../datasets/facebook/3980.edges -> N = 52
../datasets/facebook_combined.txt -> N = 4039
../datasets/p2p-Gnutella31.txt -> N = 62586
../datasets/Wiki-Vote.txt -> N = 7115
../datasets/CA-HepPh.txt -> N = 12008
../datasets/com-dblp.ungraph.txt -> N = 317080
../datasets/Email-Enron.txt -> N = 36692
../datasets/Email-EuAll.txt -> N = 265214
"""

In [6]:
datasets = ["../datasets/facebook/3980.edges",
            "../datasets/facebook/698.edges",
            "../datasets/facebook/414.edges",
            "../datasets/facebook/686.edges",
            "../datasets/facebook/348.edges",
            "../datasets/facebook/0.edges",
            "../datasets/facebook/3437.edges",
            "../datasets/facebook/1912.edges",
            "../datasets/facebook/1684.edges",
            "../datasets/facebook/107.edges",
            "../datasets/facebook_combined.txt",
            "../datasets/Wiki-Vote.txt",
            "../datasets/CA-HepPh.txt",
            "../datasets/Email-Enron.txt",
            "../datasets/p2p-Gnutella31.txt",
            "../datasets/Email-EuAll.txt",
            "../datasets/com-dblp.ungraph.txt"]
list_types = ["raw_list"]*10 + \
             ["list",
              "list",
              "raw_list",
              "raw_list",
              "list",
              "list",
              "list"]

In [7]:
idx = 0
dataset = datasets[idx]
list_type = list_types[idx]

In [8]:
# load a dataset, in both weighted and unweighted forms
A_weighted, N, original_idxs = edgelistParser(dataset, list_type, True)
A, _, _ = edgelistParser(dataset, list_type, False)

In [9]:
inDeg, _ = getInOutDegree(A_weighted)
print("Nodes:", N, "\b, AvgDeg:", np.array(inDeg).mean())

Nodes: 52, AvgDeg: 11.23076923076923


In [10]:
print("Unweighted:")
%timeit localPageRank(A, c=0.85, return_only_neighbours=True, epsilon=1e-5, max_iters=50)
print("Weigted:")
%timeit localPageRank_weight(A_weighted, c=0.85, return_only_neighbours=True, epsilon=1e-5, max_iters=50)

Unweighted:
2.45 ms ± 77.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Weigted:
2.8 ms ± 246 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
_ = """
Nodes: 52, AvgDeg: 11.23076923076923
    Unweighted:
    2.45 ms ± 77.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
    Weigted:
    2.8 ms ± 246 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



"""